In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# Đường dẫn đến ChromeDriver
service = Service(executable_path='D:/chromedriver-win32/chromedriver.exe')
driver = webdriver.Chrome(service=service)

# URL chính của Premier League Stats
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

# Truy cập trang web
driver.get(standings_url)

# Chờ cho bảng xếp hạng hiển thị
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'table.stats_table'))
)

# Lấy nội dung trang và chuyển thành BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'lxml')

# Trích xuất bảng xếp hạng
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

# Truy cập đội bóng đầu tiên
driver.get(team_urls[0])

# Chờ cho bảng "Scores & Fixtures" hiển thị
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//caption[contains(text(), 'Scores & Fixtures')]"))
)

# Lấy dữ liệu "Scores & Fixtures"
matches_soup = BeautifulSoup(driver.page_source, 'lxml')
matches = pd.read_html(driver.page_source, match="Scores & Fixtures")[0]

# Lấy link đến trang "Shooting"
links = matches_soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

# Truy cập trang Shooting
driver.get(f"https://fbref.com{links[0]}")

# Chờ cho bảng Shooting hiển thị
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//caption[contains(text(), 'Shooting')]"))
)

# Lấy dữ liệu Shooting
shooting = pd.read_html(driver.page_source, match="Shooting")[0]

# In kết quả để kiểm tra
print(matches.head())
print(shooting.head())  

# Đóng trình duyệt
driver.quit()


C:\Users\os\AppData\Local\Temp\ipykernel_14252\3082843820.py:44: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(driver.page_source, match="Scores & Fixtures")[0]
C:\Users\os\AppData\Local\Temp\ipykernel_14252\3082843820.py:60: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(driver.page_source, match="Shooting")[0]


         Date           Time            Comp         Round  Day Venue Result  \
0  2024-08-17  12:30 (18:30)  Premier League   Matchweek 1  Sat  Away      W   
1  2024-08-25  16:30 (22:30)  Premier League   Matchweek 2  Sun  Home      W   
2  2024-09-01  16:00 (22:00)  Premier League   Matchweek 3  Sun  Away      W   
3  2024-09-14  15:00 (21:00)  Premier League   Matchweek 4  Sat  Home      L   
4  2024-09-17  21:00 (02:00)    Champions Lg  League phase  Tue  Away      W   

  GF GA         Opponent   xG  xGA Poss Attendance          Captain Formation  \
0  2  0     Ipswich Town  2.6  0.5   62      30014  Virgil van Dijk   4-2-3-1   
1  2  0        Brentford  2.5  0.5   62      60017  Virgil van Dijk   4-2-3-1   
2  3  0   Manchester Utd  1.8  1.4   47      73738  Virgil van Dijk   4-2-3-1   
3  0  1  Nott'ham Forest  0.9  0.4   68      60344  Virgil van Dijk   4-2-3-1   
4  3  1         it Milan  3.1  0.6   51      59826  Virgil van Dijk   4-2-3-1   

  Opp Formation         Referee 

In [18]:
shooting.columns = shooting.columns.droplevel()

In [19]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [20]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2024-08-17,12:30 (18:30),Premier League,Matchweek 1,Sat,Away,W,2,0,Ipswich Town,...,4-2-3-1,Tim Robinson,Match Report,NaN,18,5,14.8,0.0,0,0
1,2024-08-25,16:30 (22:30),Premier League,Matchweek 2,Sun,Home,W,2,0,Brentford,...,4-4-2,Stuart Attwell,Match Report,NaN,19,8,13.6,1.0,0,0
2,2024-09-01,16:00 (22:00),Premier League,Matchweek 3,Sun,Away,W,3,0,Manchester Utd,...,4-2-3-1,Anthony Taylor,Match Report,NaN,11,3,13.4,0.0,0,0
3,2024-09-14,15:00 (21:00),Premier League,Matchweek 4,Sat,Home,L,0,1,Nott'ham Forest,...,4-2-3-1,Michael Oliver,Match Report,NaN,14,5,14.9,0.0,0,0
4,2024-09-17,21:00 (02:00),Champions Lg,League phase,Tue,Away,W,3,1,it Milan,...,4-2-3-1,Espen Eskås,Match Report,NaN,23,11,15.7,1.0,0,0


In [21]:
years = list(range(2022, 2020, -1))
all_matches = []

In [25]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)


In [26]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# Đường dẫn đến ChromeDriver
service = Service(executable_path='D:/chromedriver-win32/chromedriver.exe')
driver = webdriver.Chrome(service=service)

# Danh sách các năm
for year in years:
    # Truy cập trang xếp hạng
    standings_url = f"https://fbref.com/en/comps/9/Premier-League-Stats/{year}"
    driver.get(standings_url)

    # Chờ bảng xếp hạng hiển thị
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'table.stats_table'))
    )

    # Lấy nội dung trang và chuyển thành BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'lxml')

    # Trích xuất bảng xếp hạng
    standings_table = soup.select('table.stats_table')[0]
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    # Lấy liên kết đến mùa giải trước
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    # Lặp qua các đội bóng
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        
        # Truy cập trang đội bóng
        driver.get(team_url)

        # Chờ bảng "Scores & Fixtures" hiển thị
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//caption[contains(text(), 'Scores & Fixtures')]"))
        )

        # Lấy dữ liệu "Scores & Fixtures"
        matches_soup = BeautifulSoup(driver.page_source, 'lxml')
        matches = pd.read_html(driver.page_source, match="Scores & Fixtures")[0]

        # Lấy liên kết đến trang "Shooting"
        links = matches_soup.find_all('a')
        links = [l.get("href") for l in links]
        links = [l for l in links if l and 'all_comps/shooting/' in l]

        # Truy cập trang Shooting
        driver.get(f"https://fbref.com{links[0]}")

        # Chờ bảng Shooting hiển thị
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//caption[contains(text(), 'Shooting')]"))
        )

        # Lấy dữ liệu Shooting
        shooting = pd.read_html(driver.page_source, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            # Merge dữ liệu matches và shooting theo cột "Date"
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        
        # Chỉ giữ dữ liệu của Premier League
        team_data = team_data[team_data["Comp"] == "Premier League"]

        # Thêm thông tin về mùa giải và tên đội
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)

        # Chờ 1 giây trước khi tiếp tục (để tránh bị chặn yêu cầu quá nhanh)
        time.sleep(1)

# Đóng trình duyệt
driver.quit()


C:\Users\os\AppData\Local\Temp\ipykernel_14252\144451142.py:52: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(driver.page_source, match="Scores & Fixtures")[0]
C:\Users\os\AppData\Local\Temp\ipykernel_14252\144451142.py:68: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(driver.page_source, match="Shooting")[0]
C:\Users\os\AppData\Local\Temp\ipykernel_14252\144451142.py:52: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(driver.page_source, match="Scores & Fixtures")[0]
C:\Users\os\AppData\Local\Temp\ipykernel_14252\144451142.py:68: FutureWarning: Passi

In [27]:
len(all_matches)

78

In [28]:
match_df = pd.concat(all_matches)

In [29]:
match_df.columns = [c.lower() for c in match_df.columns]

In [30]:
match_df = pd.concat(all_matches)

In [31]:
match_df.to_csv("matchess.csv")